In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import os

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Import data loading and preprocessing utilities
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

# Define key parameters
DATA_DIR = '/kaggle/input/training/train/'
IMG_HEIGHT = 224  # Using a standard size like 224x224 is good for transfer learning
IMG_WIDTH = 224
BATCH_SIZE = 32   # Number of images to process in one batch
EPOCHS = 20       # Number of times to iterate over the entire dataset

In [8]:
# Load the dataset from the directory
# This creates a dataset of (image, label) pairs
full_dataset = image_dataset_from_directory(
    DATA_DIR,
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical' # Use 'categorical' for multi-class classification [cite: 6]
)

# Get class names
class_names = full_dataset.class_names
print("Class Names:", class_names)

# --- Create Train/Validation/Test Split ---
# We need to split the full_dataset (10,000 images)

# First, take 10% for the test set
test_batches = int(len(full_dataset) * 0.1)  # 10% for testing
test_set = full_dataset.take(test_batches)
train_val_set = full_dataset.skip(test_batches)

# From the remaining 90%, take ~11% (which is 10% of the original) for validation
val_batches = int(len(full_dataset) * 0.1) # 10% for validation
validation_set = train_val_set.take(val_batches)
train_set = train_val_set.skip(val_batches) # The rest is for training (~80%)

print(f"Total batches: {len(full_dataset)}")
print(f"Training batches: {len(train_set)}")
print(f"Validation batches: {len(validation_set)}")
print(f"Test batches: {len(test_set)}")

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size=AUTOTUNE)

Found 6787 files belonging to 10 classes.
Class Names: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Total batches: 213
Training batches: 171
Validation batches: 21
Test batches: 21


In [9]:
# Create a data augmentation layer
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)

# Create a normalization layer
normalization_layer = layers.Rescaling(1./255)

In [10]:
# Build the model
model_1 = Sequential([
    # Input layer: Apply normalization
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    normalization_layer,
    
    # Apply data augmentation ONLY during training
    data_augmentation,
    
    # CNN Body
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Classifier Head
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5), # Dropout is another key technique to reduce overfitting 
    layers.Dense(len(class_names), activation='softmax') # Softmax for multi-class
], name="custom_cnn")

# Compile the model
model_1.compile(
    optimizer='adam',
    loss='categorical_crossentropy', # Standard loss for multi-class classification [cite: 26]
    metrics=['accuracy'] # This is Top-1 Accuracy [cite: 17]
)

model_1.summary()

# Train the model
history_1 = model_1.fit(
    train_set,
    validation_data=validation_set,
    epochs=EPOCHS
)

Model: "custom_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,401,226 (169.38 MB)

 Trainable params: 44,401,226 (169.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - accuracy: 0.1779 - loss: 2.3551 - val_accuracy: 0.2545 - val_loss: 2.1090
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.2506 - loss: 2.1016 - val_accuracy: 0.2991 - val_loss: 2.0278
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 405s 2s/step - accuracy: 0.3016 - loss: 2.0043 - val_accuracy: 0.3259 - val_loss: 1.9952
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 367s 2s/step - accuracy: 0.3142 - loss: 1.9620 - val_accuracy: 0.3571 - val_loss: 1.8646
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.3534 - loss: 1.8865 - val_accuracy: 0.3720 - val_loss: 1.8195
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 388s 2s/step - accuracy: 0.3496 - loss: 1.8378 - val_accuracy: 0.4122 - val_loss: 1.7502
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 379s 2s/step - accuracy: 0.3716 - loss: 1.8233 - val_accuracy: 0.3720 - val_loss: 1.8291
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 371s 2s/step - accuracy: 0.3853 - loss: 1.8040 - val_accu

In [12]:
# 1. Load the pre-trained base model (without its top classification layer)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,  # Do NOT include the final 1000-neuron layer
    weights='imagenet'
)

# 2. Freeze the base model
base_model.trainable = False

# 3. Create our new model on top
inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = normalization_layer(inputs) # Normalize first
x = data_augmentation(x) # Apply augmentation
x = base_model(x, training=False) # Run the base model (in inference mode)
x = layers.GlobalAveragePooling2D()(x) # Pool the features
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x) # Final output layer

model_2 = keras.Model(inputs, outputs, name="transfer_learning_resnet50")

# 4. Compile the model
model_2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_2.summary()

# 5. Train the model (only the new layers)
history_2 = model_2.fit(
    train_set,
    validation_data=validation_set,
    epochs=EPOCHS
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "transfer_learning_resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,579,978 (13.66 MB)

 Trainable params: 1,321,994 (5.04 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 167s 913ms/step - accuracy: 0.4839 - loss: 1.7086 - val_accuracy: 0.7455 - val_loss: 0.8269
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 153s 894ms/step - accuracy: 0.6590 - loss: 1.0580 - val_accuracy: 0.7381 - val_loss: 0.7858
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 159s 932ms/step - accuracy: 0.6886 - loss: 0.9465 - val_accuracy: 0.7619 - val_loss: 0.7578
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 154s 900ms/step - accuracy: 0.7127 - loss: 0.8676 - val_accuracy: 0.7366 - val_loss: 0.7832
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 206s 925ms/step - accuracy: 0.7235 - loss: 0.8350 - val_accuracy: 0.7693 - val_loss: 0.7245
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 153s 893ms/step - accuracy: 0.7302 - loss: 0.8035 - val_accuracy: 0.7589 - val_loss: 0.7360
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 162s 950ms/step - accuracy: 0.7400 - loss: 0.7689 - val_accuracy: 0.7604 - val_loss: 0.7167
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 158s 927ms/step - accuracy: 0.7462 -

In [2]:
def plot_history(history, model_name):
    plt.figure(figsize=(12, 4))
    
    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{model_name} - Accuracy')
    plt.legend()
    
    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} - Loss')
    plt.legend()
    
    plt.show()

plot_history(history_1, "Custom CNN")
plot_history(history_2, "Transfer Learning (ResNet50)")

NameError: name 'history_1' is not defined

In [1]:
# Helper function to get all labels and predictions from the test set
def get_labels_and_predictions(model, test_dataset):
    y_true = []
    y_pred = []
    for images, labels in test_dataset:
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        preds = model.predict(images)
        y_pred.extend(np.argmax(preds, axis=1))
    return y_true, y_pred

# --- Model 1 Evaluation ---
print("--- Custom CNN Evaluation ---")
test_loss_1, test_acc_1 = model_1.evaluate(test_set)
print(f"Top-1 Test Accuracy: {test_acc_1:.4f}")

y_true_1, y_pred_1 = get_labels_and_predictions(model_1, test_set)
report_1 = classification_report(y_true_1, y_pred_1, target_names=class_names, output_dict=True)
avg_acc_1 = report_1['macro avg']['recall'] # In a balanced dataset, macro avg recall is avg accuracy per class
print(f"Average Accuracy per Class: {avg_acc_1:.4f}")

# --- Model 2 Evaluation ---
print("\n--- Transfer Learning (ResNet50) Evaluation ---")
test_loss_2, test_acc_2 = model_2.evaluate(test_set)
print(f"Top-1 Test Accuracy: {test_acc_2:.4f}")

y_true_2, y_pred_2 = get_labels_and_predictions(model_2, test_set)
report_2 = classification_report(y_true_2, y_pred_2, target_names=class_names, output_dict=True)
avg_acc_2 = report_2['macro avg']['recall']
print(f"Average Accuracy per Class: {avg_acc_2:.4f}")

--- Custom CNN Evaluation ---


NameError: name 'model_1' is not defined